In [12]:
from tkinter import filedialog
import tkinter as tk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import matplotlib.pyplot as plt
import joblib

In [13]:
datasets = pd.read_csv('../datasets/TrainDataFeb.csv')

In [14]:
datasets

,Time,Base_Supplier,Line,Ai_Press_RTB_Fixture,Max_RTB_Force,Ai_Press_Time,Set_Dim_A,Set_Dim_B,Set_Dim_c,Ramp_to_Datum,...,Static_Dim_A,Static_Dim_B,Static_Dim_c,Speed,Ramp_Pivot,Static_Parallelism,Spin_direction_Value,Flange_to_Pivot_Pad_Datum_F,PFH,okng_PFH
0,1675234963,2,4,1,1330,1675235026,0.5183,0.5131,0.5279,-0.846,...,0.5175,0.5048,0.5100,7358.0,8.532,14.70,7.22,0.5198,0.5482,OK
1,1675234970,2,4,0,1362,1675235031,0.5244,0.5175,0.5258,-0.858,...,0.5196,0.5154,0.5074,7393.0,8.524,14.29,6.64,0.5226,0.5369,OK
2,1675234977,2,4,1,676,1675235040,0.5102,0.5097,0.5158,-0.853,...,0.5076,0.5044,0.4998,7312.0,8.521,9.07,6.16,0.5119,0.5243,OK
3,1675234983,2,4,0,1124,1675235063,0.5099,0.4973,0.5140,-0.849,...,0.5050,0.4947,0.4965,7324.0,8.525,12.66,6.52,0.5071,0.5364,OK
4,1675234989,2,4,1,435,1675235088,0.5058,0.5075,0.5073,-0.849,...,0.5034,0.5033,0.4902,7387.0,8.524,17.57,6.12,0.5069,0.5070,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826500,1677653890,2,4,1,919,1677652590,0.5265,0.5129,0.5356,-0.855,...,0.5241,0.5085,0.5165,7233.0,8.532,18.06,6.83,0.5250,0.5664,OK
826501,1677653898,2,4,1,1373,1677654074,0.5173,0.5079,0.5257,-0.851,...,0.5149,0.5006,0.5091,7228.0,8.526,16.57,6.94,0.5170,0.5499,OK
826502,1677653904,2,4,1,808,1677653376,0.5300,0.4866,0.5115,-0.851,...,0.5257,0.4826,0.4938,7211.0,8.527,51.72,6.83,0.5094,0.5440,OK
826503,1677653913,2,4,1,919,1677652590,0.5250,0.5129,0.5359,-0.855,...,0.5235,0.5076,0.5173,7194.0,8.532,18.58,6.69,0.5246,0.5673,NG


In [15]:
from collections import Counter
print(Counter(datasets['okng_PFH']))

Counter({'OK': 817315, 'NG': 9190})


In [16]:
from imblearn.over_sampling import SMOTE
# over_sampling คือการทำค่า fail 1284 ให้เท่ากับค่า pass 271k เพื่อให้ data ในการ train เพิ่มมากขึ้น
from collections import Counter

k = 3
X = datasets.loc[:, datasets.columns != 'okng_PFH']
y = datasets.okng_PFH

# sampling_strategy=0.1 oversampling 10%
sm = SMOTE(sampling_strategy='minority', k_neighbors=k, random_state=100)
X_res, y_res = sm.fit_resample(X, y)

datasets = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)

In [17]:
datasets.columns

Index(['Time', 'Base_Supplier', 'Line', 'Ai_Press_RTB_Fixture',
       'Max_RTB_Force', 'Ai_Press_Time', 'Set_Dim_A', 'Set_Dim_B', 'Set_Dim_c',
       'Ramp_to_Datum', 'Pivot_Height', 'Parallelism', 'TIR', 'TIR_A', 'TIR_B',
       'TIR_C', 'Static_Dim', 'Static_Dim_A', 'Static_Dim_B', 'Static_Dim_c',
       'Speed', 'Ramp_Pivot', 'Static_Parallelism', 'Spin_direction_Value',
       'Flange_to_Pivot_Pad_Datum_F', 'PFH', 'okng_PFH'],
      dtype='object')

In [18]:
from pyarc import CBA, TransactionDB
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report

train, test = train_test_split(datasets, test_size=0.1)

txns_train = TransactionDB.from_DataFrame(train)
txns_test = TransactionDB.from_DataFrame(test)

cba = CBA (support = 0.07, confidence = 0.7, algorithm ='m1')
cba.fit(txns_train)

y_pred = cba.predict(txns_test)
y_test = test['okng_PFH']
print ((classification_report(y_test,y_pred)))


In [ ]:
cba.clf.rules

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

train, test = train_test_split(datasets, test_size=0.01)
print(train.shape,test.shape)

print(Counter(test['Projection1']))

txns_train = TransactionDB.from_DataFrame(train)
txns_test = TransactionDB.from_DataFrame(test)

In [ ]:
from pyarc.algorithms import (
    top_rules,
    createCARs,
    generateCARs,
    M1Algorithm,
    M2Algorithm,
)

rules = top_rules(txns_train.string_representation, minlen=2, init_maxlen=4, max_iterations=50)

# convert them to class association rules
cars = createCARs(rules)

# classifier = M1Algorithm(cars, txns_train).build()
classifier = M2Algorithm(cars, txns_train).build()

accuracy = classifier.test_transactions(txns_test)

y_pred = classifier.predict_all(txns_test)
y_test = test['Projection1']
print(classification_report(y_pred, y_test))